In [ ]:
import urllib
import os
import pandas as pd
from datetime import datetime
import glob
from io import StringIO

Створити віртуальне середовище (venv) в якому будуть встановлені всі необхідні бібліотеки та налаштування для даної лабораторної роботи;

Для кожної з адміністративних одиниць України завантажити (urllib) тестові структуровані файли, що містять значення VHI-індексу. При зберіганні файлу, до його імені потрібно додати дату та час завантаження. Передбачити повторні запуски скрипту, реалізувати механізм запобігання повторного довантаження та колізії даних;

Зчитати завантажені текстові файли у pandas dataframe. Здійснити data cleaning: прибрати зайві стовпці, заповнити пропуски, видалити зайвий текст тощо. Додати стовпчики з назвою та індексом області (див. How to Handle Missing Data in Python?)

Реалізувати процедуру зміни індексів: в завантажених з NOAA даних області індексуються за англійською абеткою (Province 1 - Cherkasy), потрібно замінити індекси так, щоб області індексувались за українською абеткою (1 область - Вінницька). 


In [165]:
foldername = "provinces"
province_map = {
     1: (22, "Черкаська"),
     2: (24, "Чернігівська"),
     3: (23, "Чернівецька"),
     4: (26, "АР Крим"),
     5: (3,  "Дніпропетровська"),
     6: (4,  "Донецька"),
     7: (8,  "Івано-Франківська"),
     8: (19, "Харківська"),
     9: (20, "Херсонська"),
    10: (21, "Хмельницька"),
    11: (9,  "Київська"),
    12: (25, "м. Київ"),
    13: (10, "Кіровоградська"),
    14: (11, "Луганська"),
    15: (12, "Львівська"),
    16: (13, "Миколаївська"),
    17: (14, "Одеська"),
    18: (15, "Полтавська"),
    19: (16, "Рівненська"),
    20: (27, "м. Севастополь"),
    21: (17, "Сумська"),
    22: (18, "Тернопільська"),
    23: (6,  "Закарпатська"),
    24: (1,  "Вінницька"),
    25: (2,  "Волинська"),
    26: (7,  "Запорізька"),
    27: (5,  "Житомирська"),
}

In [ ]:
def downloadprovince(id: int):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={id}&year1=1981&year2=2024&type=Mean"
    timestamp = datetime.now().strftime("%Y%m%d_%Hh%Mm%Ss")
    filename = f"Province{id}_{timestamp}.csv"
    ukr_index, ukr_name = province_map[id]
    filenameukr = f"{ukr_index}_область_{ukr_name}_{timestamp}.csv"
    #filepath = os.path.join(foldername, filename)
    filepathukr = os.path.join(foldername, filenameukr)

    try:
        print(f"[INFO] Завантаження provinceID={id}...")
        #urllib.request.urlretrieve(url, filepath)
        urllib.request.urlretrieve(url, filepathukr)
        print(f"[OK] Збережено: {filepathukr}")
    except Exception as e:
        print(f"[ERROR] Не вдалося завантажити provinceID={id}: {e}")

In [195]:
for id in range(1, 28):
    downloadprovince(id)

[INFO] Завантаження provinceID=1...
[OK] Збережено: provinces\Province1_20250929_21h43m58s.csv
[INFO] Завантаження provinceID=2...
[OK] Збережено: provinces\Province2_20250929_21h44m01s.csv
[INFO] Завантаження provinceID=3...
[OK] Збережено: provinces\Province3_20250929_21h44m03s.csv
[INFO] Завантаження provinceID=4...
[OK] Збережено: provinces\Province4_20250929_21h44m05s.csv
[INFO] Завантаження provinceID=5...
[OK] Збережено: provinces\Province5_20250929_21h44m07s.csv
[INFO] Завантаження provinceID=6...
[OK] Збережено: provinces\Province6_20250929_21h44m10s.csv
[INFO] Завантаження provinceID=7...
[OK] Збережено: provinces\Province7_20250929_21h44m12s.csv
[INFO] Завантаження provinceID=8...
[OK] Збережено: provinces\Province8_20250929_21h44m14s.csv
[INFO] Завантаження provinceID=9...
[OK] Збережено: provinces\Province9_20250929_21h44m16s.csv
[INFO] Завантаження provinceID=10...
[OK] Збережено: provinces\Province10_20250929_21h44m18s.csv
[INFO] Завантаження provinceID=11...
[OK] Збереж

In [135]:
files = glob.glob("provinces/Province1_*.csv")
if files:
    latestfile = sorted(files)[-1]
    print(f"Читаю файл: {latestfile}")

    with open(latestfile, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    lines = [line.replace('<tt><pre>', '').replace('</pre></tt>', '') for line in lines]
    data_lines = [line for line in lines if  not line.startswith('Mean') and not line.startswith('year')]

    data_lines = [line.rstrip().rstrip(',')+ '\n'  for line in data_lines] #якщо закоментувати, буде зайва кома та відповідно зайва колонка

    data_csv = ''.join(data_lines)
    #print(data_csv) #просто як виглядає один рядок

    columns = ['year', 'week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']
    df = pd.read_csv(StringIO(data_csv), names=columns)

    #df = df.dropna(how='all')

    print(df.head())

Читаю файл: provinces\Province1_20250929_19h00m11s.csv
   year  week    SMN     SMT    VCI    TCI    VHI
0  1982     1  0.053  260.31  45.01  39.46  42.23
1  1982     2  0.054  262.29  46.83  31.75  39.29
2  1982     3  0.055  263.82  48.13  27.24  37.68
3  1982     4  0.053  265.33  46.09  23.91  35.00
4  1982     5  0.050  265.66  41.46  26.65  34.06


In [ ]:
#files = glob.glob("provinces/Province1_*.csv")
files = glob.glob("provinces/1_область_Вінницька_*.csv")
if files:
    latestfile = sorted(files)[-1]
    print(f"Читаю файл: {latestfile}")
    columns = ['year', 'week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']
    #with open(latestfile, "r", encoding="utf-8") as f:
    #    text = f.read()
    #text = text.replace('<tt><pre>', '').replace('</pre></tt>', '')
    #df = pd.read_csv(StringIO(text),  skiprows=2, names=columns , header=None, sep=",")
    
    df= pd.read_csv(latestfile,header=None, skiprows=2, comment='<')
    del df[7]
    df.columns = columns
    print(df.head())
    #df.info()

Читаю файл: provinces\1_область_Вінницька_20250929_21h01m53s.csv
   year  week    SMN     SMT    VCI    TCI    VHI
0  1982     2  0.074  265.78  67.62  23.05  45.34
1  1982     3  0.076  267.19  69.37  20.40  44.88
2  1982     4  0.075  268.57  65.26  17.93  41.60
3  1982     5  0.072  269.24  58.58  20.00  39.29
4  1982     6  0.071  270.12  52.37  22.93  37.65


In [187]:
def df_by_index(ProvinceIndex):
    ukr_index, ukr_name = province_map[ProvinceIndex]
    files = glob.glob(f"provinces/{ukr_index}_область_{ukr_name}_*.csv")
    if files:
        latestfile = sorted(files)[-1]
        columns = ['year', 'week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']
        df= pd.read_csv(latestfile,header=None, skiprows=2, comment='<')
        del df[7]
        df.columns = columns
    return df

Ряд VHI для області за вказаний рік

In [190]:
def vhi_by_year(df, year):
    return df[(df["year"] == year)]["VHI"]


vhi_by_year(df_by_index(24), 2014)


1663    47.80
1664    50.45
1665    53.22
1666    52.93
1667    49.86
1668    45.95
1669    42.61
1670    41.32
1671    43.25
1672    46.73
1673    45.47
1674    47.70
1675    47.97
1676    48.01
1677    50.72
1678    54.11
1679    56.74
1680    59.08
1681    59.63
1682    61.25
1683    63.55
1684    63.46
1685    64.02
1686    66.27
1687    66.94
1688    66.10
1689    65.68
1690    65.09
1691    63.69
1692    61.58
1693    59.75
1694    58.45
1695    56.98
1696    54.03
1697    50.75
1698    48.39
1699    47.01
1700    44.70
1701    43.11
1702    43.86
1703    45.40
1704    43.66
1705    39.79
1706    36.37
1707    38.54
1708    39.42
1709    40.14
1710    43.47
1711    46.69
1712    47.18
1713    46.61
1714    46.13
Name: VHI, dtype: float64

Ряд VHI за вказаний діапазон років для вказаних областей

In [191]:
def vhi_by_range(df, y1, y2):
    return df[(df["year"].between(y1, y2))]


vhi_by_range(df_by_index(24), 2000, 2014)

,year,week,SMN,SMT,VCI,TCI,VHI
935,2000,1,0.023,260.25,8.46,39.97,24.22
936,2000,2,0.023,259.38,10.27,45.13,27.70
937,2000,3,0.024,259.61,14.76,46.60,30.68
938,2000,4,0.027,260.21,19.52,45.57,32.55
939,2000,5,0.030,260.59,22.24,47.21,34.73
...,...,...,...,...,...,...,...
1710,2014,48,0.081,267.09,42.44,44.50,43.47
1711,2014,49,0.074,263.81,44.83,48.55,46.69
1712,2014,50,0.071,261.75,49.50,44.87,47.18
1713,2014,51,0.068,260.61,53.34,39.88,46.61


Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани

In [194]:
def vhi_stats(df,y1, y2):
    subset = df[(df["year"].between(y1, y2))]
    return {
        "min": subset["VHI"].min().item(),
        "max": subset["VHI"].max().item(),
        "mean": subset["VHI"].mean().item(),
        "median": subset["VHI"].median().item()
    }

vhi_stats(df_by_index(24),2000,2014)

{'min': -1.0, 'max': 81.44, 'mean': 47.75448717948718, 'median': 48.23}

In [ ]:
#files = glob.glob("provinces/Province8_*.csv")
#latestfile = sorted(files)[-1]  
